# How To use FastAI LRFinder with Ignite

This example demonstrates the usage of `FastaiLRFinder` with a 3-conv. layer network on the MNIST dataset.

## Required Dependencies

We assume that `torch` and `ignite` are already installed. We can install it using pip:

## Basic Setup

For this example we'll be using MNIST for our dataset, a ResNet18 model and other hyperparameters as used in the [Getting Started Notebook]().

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets import MNIST

from ignite.engine import create_supervised_trainer, create_supervised_evaluator
from ignite.metrics import Loss, Accuracy
from ignite.handlers import FastaiLRFinder

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])

trainset = MNIST(mnist_pwd, train=True, download=True, transform=transform)
trainloader = DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=4)

testset = MNIST(mnist_pwd, train=False, download=True, transform=transform)
testloader = DataLoader(testset, batch_size=batch_size * 2, shuffle=False, num_workers=0)


class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        
        # Changed the output layer to output 10 classes instead of 1000 classes
        self.model = resnet18(num_classes=10)

        # Changed the input layer to take grayscale images for MNIST instaed of RGB images
        self.model.conv1 = nn.Conv2d(1, 64, kernel_size=(3, 3), stride=(2, 2), padding=(3, 3), bias=False)

    def forward(self, x):
        return self.model(x)


model = Net().to(device)
optimizer = torch.optim.RMSprop(model.parameters(), lr=0.005)
criterion = nn.CrossEntropyLoss()

## Training loss (fastai)

This learning rate test range follows the same procedure used by fastai. The model is trained for `num_iter` iterations while the learning rate is increased from its initial value specified by the optimizer algorithm to `end_lr`. The increase can be linear (`step_mode="linear"`) or exponential (`step_mode="exp"`); linear provides good results for small ranges while exponential is recommended for larger ranges.

In [ ]:
# save the initial state for the model and the optimizer
# to reset them later
init_model_state = model.state_dict()
init_opt_state = optimizer.state_dict()

In [ ]:
trainer = create_supervised_trainer(model, optimizer, criterion, device=device)

lr_finder = FastaiLRFinder()
to_save={'model': model, 'optimizer': optimizer}
with lr_finder.attach(trainer, to_save, diverge_th=1.5) as trainer_with_lr_finder:
    trainer_with_lr_finder.run(trainloader)

Let's plot the found optimal learning rate.

In [ ]:
%matplotlib inline

from matplotlib import pyplot as plt
ax = lr_finder.plot()
plt.show()

print("Suggested LR", lr_finder.lr_suggestion())

*Note, that model and optimizer were restored to their initial states when `FastaiLRFinder` finished.*

Let's compare training results: 
- a) using a fixed learning rate (`lr=3e-4`)
- b) with applying optimal learning rate.

In [ ]:
# with lr=3e-4
trainer.run(trainloader, max_epochs=5)

evaluator = create_supervised_evaluator(model, metrics={"acc": Accuracy(), "loss": Loss(nn.NLLLoss())}, device=device)
evaluator.run(testloader)

print(evaluator.state.metrics)

After training we need to get the model and the optimizer back to their initial state, to do that, we will load the initial state again.

In [ ]:
# Reinit model / optimizer
model.load_state_dict(init_model_state)
optimizer.load_state_dict(init_opt_state)

Let's now apply suggested learning rate to the optimizer, and train the model again with optimal learning rate.

In [ ]:
lr_finder.apply_suggested_lr(optimizer)
print(optimizer.param_groups[0]['lr'])

In [ ]:
trainer.run(trainloader, max_epochs=5)

evaluator.run(testloader)
print(evaluator.state.metrics)